Utility Funcitons for loading and handling processed data

In [28]:
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
from pyspark import SparkContext
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)

NameError: name 'sc' is not defined

In [29]:
sc = SparkContext('local[*]','temp')
sqlContext=SQLContext(sc)

22/07/23 03:07:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/23 03:07:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [30]:
dt=np.dtype([('id',np.ulonglong),('start year',np.short),('start month,',np.byte),('start day',np.byte),('start hour',np.byte),('start minute',np.byte),('end year',np.short),('end month',np.byte),('end day',np.byte),('end hour',np.byte),('end minute',np.byte),('trip seconds',np.uint32),('trip miles',np.single),('pickup census tract',np.ulonglong),('dropoff census tract',np.ulonglong),('pickup community area',np.byte),('dropoff community area',np.byte),('fare',np.single),('tip',np.single),('addcharge',np.single),('trip total',np.single),('st auth',np.byte),('pool',np.byte),('pickup lat',np.double),('pickup lon',np.double),('dropoff lat',np.double),('dropoff lon',np.double)])

In [31]:
def read_array(x):
    array=np.frombuffer(bytes(x),dtype=dt)
    return array.tolist()[0]

In [37]:
def read_in_range(start_year,start_month,end_year,end_month,path='/media/benjamin/Data/Chicago_Transit/TNP/'):
    #Calculate month and year ranges
    nmonths=(end_year-start_year-1)*12.+(12.-start_month+1)+end_month
    yrs=(start_year+np.floor((start_month+np.arange(0,nmonths)-1)/12.)).astype(int)
    mos=(np.floor((np.mod(start_month+np.arange(nmonths)-1,12)+1 ))).astype(int)
    
    #construct list of file names
    fnames=''
    if nmonths > 1:
        for i in np.arange(0,nmonths,dtype=np.int16):
            fnames+=(path+"{0:04d}{1:02d}".format(yrs[i],mos[i])+'TNP.dat')
            if i != nmonths-1:
                fnames+=','
    else:
        fnames=path+"{0:04d}{1:02d}".format(yrs,mos)+'TNP.dat'
        
    #load binary records into rdd given record length
    rdd=sc.binaryRecords(fnames,96)
    out=rdd.map(read_array)
    df=out.toDF(['id','start year','start month','start day','start hour','start minute','end year','end month','end day','end hour','end minute','trip seconds','trip miles','pickup census tract','dropoff census tract','pickup community area','dropoff community area','fare','tip','addcharge','trip total','st auth','pool','pickup lat','pickup lon','dropaff lat','dropoff lon'])
    out.unpersist()
    return df

In [42]:
def area_out(df,ca):
    with_out=df.groupby(['start year','start month','start day','start hour','pickup community area']).count()
    out_area=with_out.where(with_out['pickup community area']==ca)
    outdf=out_area.sort(['start year','start month','start day','start hour']).toPandas()
    return outdf

In [38]:
df=read_in_range(2019,11,2019,12)

In [43]:
outdf=area_out(df,32)

In [44]:
outdf.head()

,start year,start month,start day,start hour,pickup community area,count
0,2019,11,1,0,32,676
1,2019,11,1,1,32,334
2,2019,11,1,2,32,234
3,2019,11,1,3,32,113
4,2019,11,1,4,32,138
